# Bias in word embeddings
Word embeddings is a technique in NLP and text mining to represent words in order to be able to compare words based on similarity. The word embeddings are trained on large data sets with text written by humans. Therefore, the bias we have and include in our writungs will be transferred to the word embeddings. This project looks at what bias there are in the swedish word embeddings "Swectors" based on text from Göteborgsposten. They will also be compared with another set of word ebedding that will be trained on antoher set of data.

In [1]:
import bz2
import pandas as pd
import numpy as np
import time

## Importing datasets
Importing the large set of vectors might take a little while, please be patient.

In [11]:
colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]

with bz2.open("swectors-300dim.txt.bz2") as source:
    swectors = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
with bz2.open("becctors-300dim.txt.bz2") as source:
    becctors = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
#swectors.tail()

### Import small sized dataset

colnames = ["word"] + ["dim" + str(x) for x in range(1,301)]
with bz2.open("swectors_short-300dim.txt.bz2") as source:
    swectors_short = pd.read_csv(source, header=None, names=colnames, delimiter=" ", skiprows=[0])
    
swectors_short.tail()

Extract the vector for the word 'kvinna', in order to look att similar words for bias measures.
Get the 300 dimensions from the dataframe, convert to numpy and get the following format: `[[dim1 dim2 ... dim299 dim300]]`, take the first element to get a single list. Save it as a tuple with word first and vector second.

In [12]:
kvinna_s = ('kvinna', swectors.loc[swectors['word'] == 'kvinna'].loc[:, 'dim1':'dim300'].to_numpy()[0])
man_s = ('man', swectors.loc[swectors['word'] == 'man'].loc[:, 'dim1':'dim300'].to_numpy()[0])

kvinna_b = ('kvinna', becctors.loc[becctors['word'] == 'kvinna'].loc[:, 'dim1':'dim300'].to_numpy()[0])
man_b = ('man', becctors.loc[becctors['word'] == 'man'].loc[:, 'dim1':'dim300'].to_numpy()[0])

In [13]:
def cosine_similarity(word1, word2):
    # Takes two vectors and calculates the cosine similarity between them
    # @ is dot product
    v1 = word1[1]
    v2 = word2[1]
    return (v1 @ v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

In [17]:
cosine_similarity(kvinna_s, man_s)

0.3283947788176963

## Applying function to whole dataframe
This way of doing the calculation will apply a function to each row of the dataframe, and return a Series of same length with all results in it.

In [46]:
def n_most_similar(vectors, n, word, vectors_filtered=None):
    
    if vectors_filtered is not None:
        df = vectors_filtered
        print("filtered")
    else:
        df = vectors
        print("not filtered")
        
    word_vec = (word, vectors.loc[vectors['word'] == word].loc[:, 'dim1':'dim300'].to_numpy()[0])
    
    def similarity(row):
        row_vec = (row['word'], row.loc['dim1':'dim300'].to_numpy())
        return cosine_similarity(word_vec, row_vec)

    start = time.time()
    similarities = df.apply(similarity, axis=1)

    # Concatenate the top n words (plus the word itself) to the similarity values of each word.
    # Also set the correct coulmn name.
    s1 = df.loc[similarities.nlargest(n+1).index, 'word']
    s2 = similarities.nlargest(n+1)
    similars = pd.concat([s1, s2], axis=1).rename(columns={0: "similarity"})
    end = time.time()
    print("Time elapsed: ", end - start)
    return similars

In [44]:
similars_kvinna_s = n_most_similar(swectors, 10, 'kvinnan')
print(similars_kvinna_s)

not filtered
Time elapsed:  36.43729591369629
           word  similarity
588     kvinnan    1.000000
303      mannen    0.865082
1829    flickan    0.843521
2486     mamman    0.750042
2516     pojken    0.744725
4381     offret    0.719690
2775     pappan    0.679648
493      kvinna    0.679172
31          hon    0.664614
1077     männen    0.662218
34234  vårdaren    0.652844


In [32]:
similars_kvinna_b = n_most_similar(becctors, 10, 'kvinnan')
print(similars_kvinna_b)

not filtered
Time elapsed:  29.458340406417847
            word  similarity
1247     kvinnan    1.000000
5635    kvinnans    0.660167
723       kvinna    0.631024
2500     flickan    0.624874
2021      mamman    0.582821
523       mannen    0.569707
522      kvinnor    0.568113
3443   kvinnorna    0.565799
14527    hustrun    0.541186
971       tjejen    0.526948
4226       damen    0.517224


In [29]:
similars_män_s = n_most_similar(swectors, 10, 'mannen')
print(similars_män_s)

not filtered
Time elapsed:  36.482909202575684
                 word  similarity
303            mannen    1.000000
588           kvinnan    0.865082
2516           pojken    0.824330
9286      polismannen    0.760309
1829          flickan    0.751662
4506   gärningsmannen    0.724007
19807       ynglingen    0.708831
4381           offret    0.707234
2775           pappan    0.689243
16049          vakten    0.685423
18040      målsägaren    0.676460


In [30]:
similars_män_b = n_most_similar(becctors, 10, 'mannen')
print(similars_män_b)

not filtered
Time elapsed:  26.49137544631958
         word  similarity
523    mannen    1.000000
914     maken    0.741162
525    sambon    0.700235
759    killen    0.671338
1480   gubben    0.659938
4075    karln    0.611537
5725     frun    0.608822
32        han    0.605173
1247  kvinnan    0.569707
2297   pappan    0.562663
175     honom    0.557921


## Filtering out all adjectives in the word embeddings
To see bias in adjectives, the dataframe with the swectors is filtered to only keep words that are in the dataframe with adjectves from Språkrådet.

In [34]:
with bz2.open("adjektiv.txt.bz2") as source:
    adjectives = pd.read_csv(source)
    
swectors_filtered = swectors.loc[swectors['word'].isin(adjectives['Word'])]

becctors_filtered = becctors.loc[becctors['word'].isin(adjectives['Word'])]

In [ ]:
swectors_filtered.shape

In [61]:
similars_kvinna_adj_s = n_most_similar(swectors, 10, 'dator', swectors_filtered)
print(similars_kvinna_adj_s)

filtered
Time elapsed:  3.014665126800537
                     word  similarity
22352              bärbar    0.500918
15547             bärbara    0.491010
13986            digitalt    0.482039
21152            trådlöst    0.477740
109616          sladdlösa    0.452600
26434            trådlösa    0.433673
72740            portabla    0.404584
34138             trådlös    0.403282
12732        elektroniskt    0.399799
49645   barnpornografiska    0.395040
80051           stationär    0.394396


In [62]:
similars_kvinna_adj_b = n_most_similar(becctors, 10, 'dator', becctors_filtered)
print(similars_kvinna_adj_b)

filtered
Time elapsed:  1.7819604873657227
                word  similarity
14512        externa    0.428106
25263         mobilt    0.382808
35398       portabel    0.299629
86820     okrypterad    0.294558
119102    okrypterat    0.290431
7494        manuellt    0.283486
81643      urkopplad    0.280656
22224   elektroniskt    0.274290
5324          trasig    0.268814
23956         mobila    0.263520
111032   okrypterade    0.262835
